In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
chicago_weather = pd.read_csv("chicago_weather.csv")
chicago_trips = pd.read_csv("Chicago_Trips.csv")

In [3]:
chicago_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8783 entries, 0 to 8782
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   app_temp         8783 non-null   float64
 1   azimuth          8783 non-null   float64
 2   clouds           8783 non-null   int64  
 3   datetime         8783 non-null   object 
 4   dewpt            8783 non-null   float64
 5   dhi              8783 non-null   int64  
 6   dni              8783 non-null   int64  
 7   elev_angle       8783 non-null   float64
 8   ghi              8783 non-null   int64  
 9   h_angle          0 non-null      float64
 10  pod              8783 non-null   object 
 11  precip           8783 non-null   float64
 12  pres             8783 non-null   int64  
 13  revision_status  8783 non-null   object 
 14  rh               8783 non-null   int64  
 15  slp              8783 non-null   int64  
 16  snow             8783 non-null   float64
 17  solar_rad     

# 1 Match Time Period of Datasets
Before preprocessing the weather data, we need to verify the time coverage to ensure compatibility when merging it with other datasets.

In [4]:
# display time period of our trip data
chicago_trips['Trip End Timestamp'] = pd.to_datetime(chicago_trips['Trip End Timestamp'])
trip_time = chicago_trips.sort_values(by='Trip End Timestamp').reset_index(drop=True)

print("Trip Data Time Period:")
print("\t Earliest Date Time:", trip_time['Trip End Timestamp'].iloc[0])
print("\t Latest Date Time:", trip_time['Trip End Timestamp'].iloc[-1])

# validate the time period of the weather dataset
chicago_weather['timestamp_utc'] = pd.to_datetime(chicago_weather['timestamp_utc'])
df_chicago_temp = chicago_weather.sort_values(by='timestamp_utc').reset_index(drop=True)

print("\nWeather Data Time Period:")
print("\t Earliest Date Time:", df_chicago_temp['timestamp_utc'].iloc[0])
print("\t Latest Date Time:", df_chicago_temp['timestamp_utc'].iloc[-1])



Trip Data Time Period:
	 Earliest Date Time: 2024-01-01 00:00:00
	 Latest Date Time: 2025-04-01 19:15:00

Weather Data Time Period:
	 Earliest Date Time: 2024-01-01 06:00:00
	 Latest Date Time: 2025-01-01 05:00:00


In [5]:
chicago_weather.head()

app_temp  azimuth  clouds       datetime  dewpt  dhi  dni  elev_angle  ghi  \
0      -5.7      4.5      87  2024-01-01:06   -1.5    0    0       -71.1    0   
1      -4.9     41.5     100  2024-01-01:07   -1.1    0    0       -66.6    0   
2      -4.3     64.3      87  2024-01-01:08   -0.3    0    0       -57.7    0   
3      -5.2     78.9     100  2024-01-01:09   -0.5    0    0       -47.1    0   
4      -4.8     89.9      87  2024-01-01:10   -0.7    0    0       -36.0    0   

   h_angle  ...      timestamp_local       timestamp_utc          ts   uv  \
0      NaN  ...  2024-01-01 00:00:00 2024-01-01 06:00:00  1704088800  0.0   
1      NaN  ...  2024-01-01 01:00:00 2024-01-01 07:00:00  1704092400  0.0   
2      NaN  ...  2024-01-01 02:00:00 2024-01-01 08:00:00  1704096000  0.0   
3      NaN  ...  2024-01-01 03:00:00 2024-01-01 09:00:00  1704099600  0.0   
4      NaN  ...  2024-01-01 04:00:00 2024-01-01 10:00:00  1704103200  0.0   

   vis                                            weather  wind_dir  \
0   16  {'description': 'Overcast clouds', 'code': 804...       320   
1   16  {'description': 'Overcast clouds', 'code': 804...       330   
2   10  {'description': 'Overcast clouds', 'code': 804...       340   
3   10  {'description': 'Overcast clouds', 'code': 804...       330   
4   10  {'description': 'Overcast clouds', 'code': 804...       330   

   wind_gust_spd  wind_spd        date  
0           12.4       8.8  2024-01-01  
1           10.5       6.7  2024-01-01  
2           10.8       7.7  2024-01-01  
3           10.6       8.8  2024-01-01  
4            9.6       7.2  2024-01-01  

[5 rows x 29 columns]

In [6]:
# Step 1: Parse timestamp column as timezone-aware UTC
chicago_weather['timestamp_utc'] = pd.to_datetime(chicago_weather['timestamp_utc'], utc=True)

# Step 2: Floor to the nearest hour
floored_hours = chicago_weather['timestamp_utc'].dt.floor('H')

# Step 3: Generate full set of expected hourly timestamps (also UTC-aware)
start_time = floored_hours.min()
end_time = floored_hours.max()
expected_hours = pd.date_range(
    start=start_time,
    end=end_time,
    freq='H',
    tz='UTC'
)

# Step 4: Convert to string for set comparison
expected_hour_strs = set(expected_hours.strftime('%Y-%m-%dT%H:%M:%S%z'))
observed_hour_strs = set(floored_hours.dt.strftime('%Y-%m-%dT%H:%M:%S%z'))

# Step 5: Identify missing hours
missing_hour_strs = sorted(expected_hour_strs - observed_hour_strs)

# Step 6: Convert back to UTC datetime
missing_hours = pd.to_datetime(missing_hour_strs, format='%Y-%m-%dT%H:%M:%S%z')

# Step 7: Output
print("Time period checked:")
print(f"From: {start_time} to {end_time}\n")

print("Missing hourly timestamps (UTC):")
print(missing_hours)

print(f"\nTotal missing hours: {len(missing_hours)}")


Time period checked:
From: 2024-01-01 06:00:00+00:00 to 2025-01-01 05:00:00+00:00

Missing hourly timestamps (UTC):
DatetimeIndex(['2024-11-03 07:00:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)

Total missing hours: 1


We identified three different locations of missing entries:
- from 2024-01-01 00:00 to 05:00, filled with weather data of 2024-01-01 06:00 (earliest datapoint we have)
- 2024-11-03 07:00, filled with weather data of 2024-11-03 06:00 (data of one hour earlier)
- from 2025-01-01 06:00 to 14:00, filled with weather data of 2025-01-01 05:00 (latest datapoint we have)

In [7]:
# Ensure datetime is parsed correctly
chicago_weather['timestamp_utc'] = pd.to_datetime(chicago_weather['timestamp_utc'], utc=True)

# remove default indexing
chicago_weather = chicago_weather.reset_index(drop=True)

# Set timestamp as index
chicago_weather = chicago_weather.set_index('timestamp_utc')

# --------------------------------------------
# Fill missing rows from 2024-01-01 00:00 to 05:00
# --------------------------------------------

ref_time_1 = pd.Timestamp("2024-01-01 06:00:00", tz='UTC')
reference_row_1 = chicago_weather.loc[ref_time_1]

new_times_1 = pd.date_range(start="2024-01-01 00:00:00", end="2024-01-01 05:00:00", freq='H', tz='UTC')
new_rows_1 = pd.DataFrame([reference_row_1] * len(new_times_1), index=new_times_1)

# --------------------------------------------
# Fill missing row: 2024-11-03 07:00 using 06:00
# --------------------------------------------

ref_time_2 = pd.Timestamp("2024-11-03 06:00:00", tz='UTC')
target_time_2 = pd.Timestamp("2024-11-03 07:00:00", tz='UTC')

reference_row_2 = chicago_weather.loc[ref_time_2]
reference_row_2.name = target_time_2
new_rows_2 = pd.DataFrame([reference_row_2], index=[target_time_2])

# --------------------------------------------
# Add rows from 2025-01-01 05:00 to 14:00 using last known row of that day
# --------------------------------------------

# Find last available timestamp on 2025-01-01
jan1_rows = chicago_weather.loc["2025-01-01"]
latest_time = jan1_rows.index.max()
reference_row_3 = chicago_weather.loc[latest_time]

# Fix timezone mismatch issue explicitly
start_time = (latest_time + pd.Timedelta(hours=1)).tz_convert('UTC')
end_time = pd.Timestamp("2025-01-01 14:00:00", tz='UTC')

new_times_3 = pd.date_range(start=start_time, end=end_time, freq='H')
new_rows_3 = pd.DataFrame([reference_row_3] * len(new_times_3), index=new_times_3)

# --------------------------------------------
# Combine all new rows with original DataFrame
# --------------------------------------------

df_combined = pd.concat([chicago_weather, new_rows_1, new_rows_2, new_rows_3])
df_combined = df_combined[~df_combined.index.duplicated(keep='first')]
df_combined = df_combined.sort_index().reset_index().rename(columns={'index': 'timestamp_utc'})

# Store final result back
chicago_weather = df_combined

# Show summary
chicago_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8799 entries, 0 to 8798
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   timestamp_utc    8799 non-null   datetime64[ns, UTC]
 1   app_temp         8799 non-null   float64            
 2   azimuth          8799 non-null   float64            
 3   clouds           8799 non-null   int64              
 4   datetime         8799 non-null   object             
 5   dewpt            8799 non-null   float64            
 6   dhi              8799 non-null   int64              
 7   dni              8799 non-null   int64              
 8   elev_angle       8799 non-null   float64            
 9   ghi              8799 non-null   int64              
 10  h_angle          0 non-null      float64            
 11  pod              8799 non-null   object             
 12  precip           8799 non-null   float64            
 13  pres             8

In [8]:
# Step 1: Parse to timezone-aware UTC
# chicago_weather['timestamp_utc'] = pd.to_datetime(chicago_weather['timestamp_utc'], utc=True)

# Step 2: Floor to the hour
floored_hours = chicago_weather['timestamp_utc'].dt.floor('H')

# Step 3: Generate expected full set of hourly timestamps
expected_hours = pd.date_range(start=floored_hours.min(),
                               end=floored_hours.max(),
                               freq='H',
                               tz='UTC')

# Step 4: Compare using ISO format strings to avoid formatting issues
expected_hour_strs = set(expected_hours.strftime('%Y-%m-%dT%H:%M:%S'))
observed_hour_strs = set(floored_hours.dt.strftime('%Y-%m-%dT%H:%M:%S'))

# Step 5: Identify missing hour strings
missing_hour_strs = sorted(expected_hour_strs - observed_hour_strs)

# Step 6: Convert back to timezone-aware datetime
missing_hours = pd.to_datetime(missing_hour_strs)

# Step 7: Output the result
print("Missing hourly timestamps (UTC):")
for ts in missing_hours:
    print(ts)

print(f"\nTotal missing hours: {len(missing_hours)}")


Missing hourly timestamps (UTC):

Total missing hours: 0


# 2. General Data Cleanup

In [9]:
# check for duplicates
n_duplicates = chicago_weather.duplicated().sum()
print(f"Number of duplicate rows: {n_duplicates}")

Number of duplicate rows: 0


In [10]:
# check for NA values
# Check for NA values and include relative share
for col in chicago_weather.columns:
    null_count = chicago_weather[col].isna().sum()
    total_rows = len(chicago_weather)
    null_share = (null_count / total_rows) * 100
    print(f'Number of null values in column "{col}": {null_count} ({null_share:.2f}%)')


Number of null values in column "timestamp_utc": 0 (0.00%)
Number of null values in column "app_temp": 0 (0.00%)
Number of null values in column "azimuth": 0 (0.00%)
Number of null values in column "clouds": 0 (0.00%)
Number of null values in column "datetime": 0 (0.00%)
Number of null values in column "dewpt": 0 (0.00%)
Number of null values in column "dhi": 0 (0.00%)
Number of null values in column "dni": 0 (0.00%)
Number of null values in column "elev_angle": 0 (0.00%)
Number of null values in column "ghi": 0 (0.00%)
Number of null values in column "h_angle": 8799 (100.00%)
Number of null values in column "pod": 0 (0.00%)
Number of null values in column "precip": 0 (0.00%)
Number of null values in column "pres": 0 (0.00%)
Number of null values in column "revision_status": 0 (0.00%)
Number of null values in column "rh": 0 (0.00%)
Number of null values in column "slp": 0 (0.00%)
Number of null values in column "snow": 0 (0.00%)
Number of null values in column "solar_rad": 0 (0.00%)
Nu

# 3. Column Selection

In [11]:
chicago_weather.columns

Index(['timestamp_utc', 'app_temp', 'azimuth', 'clouds', 'datetime', 'dewpt',
       'dhi', 'dni', 'elev_angle', 'ghi', 'h_angle', 'pod', 'precip', 'pres',
       'revision_status', 'rh', 'slp', 'snow', 'solar_rad', 'temp',
       'timestamp_local', 'ts', 'uv', 'vis', 'weather', 'wind_dir',
       'wind_gust_spd', 'wind_spd', 'date'],
      dtype='object')

As we can see, there are a lot of columns in this dataset. Although the dataset provides detailed weather information for Chicago, not all variables are relevant to our specific use case. Therefore, we decided to keep the following columns and drop all other:
Temperature data:
- timestamp_utc (we can get other time related columns if needed from this column --> dropped redundant information)
- temp (temperature)
- app_temp (apparent temperature)
- rh (relative humidity)
  
Wind data:
- wind_spd
- wind_gust_spd (wind gust speed, basically highest wind speed)

Percipitation & Snow data:
- precip (Precipitation/Rainfall amount)
- snow (Snowfall amount)
  
Sun data:
- ghi (solar radiation, perception of intensity of the sun)
- vis (visibility)

In [12]:
chicago_weather = chicago_weather[["timestamp_utc", "temp", "app_temp", "rh", "wind_spd", "wind_gust_spd", "precip", "snow", "ghi", "vis"]]

# 4. Outlier Detection

In [13]:

# 2 plots per row
rows = math.ceil(len(chicago_weather.columns) / 2)

fig, axes = plt.subplots(rows, 2, figsize=(12, rows * 4))
axes = axes.flatten()

for i, col in enumerate(chicago_weather.columns):
    data = pd.to_numeric(chicago_weather[col], errors='coerce').dropna()
    axes[i].boxplot(data, vert=True)
    axes[i].set_title(f'Boxplot of {col}')
    axes[i].set_xlabel(col)
    axes[i].grid(True)

# Remove any unused axes
for j in range(len(chicago_weather.columns), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


We cannot identify any noticeable outliers. Therefore we are ready to merge the cleaned weather dataset with the cleaned trip dataset.

# 5 Merge Trip and Weather Data

In [14]:
# 1. Ensure timestamps are parsed correctly
chicago_weather['timestamp_utc'] = pd.to_datetime(chicago_weather['timestamp_utc'], utc=True)
chicago_trips['Trip Start Timestamp'] = pd.to_datetime(chicago_trips['Trip Start Timestamp'])

# 2. Localize trip timestamps to UTC if they are naive (no tz info)
if chicago_trips['Trip Start Timestamp'].dt.tz is None:
    chicago_trips['Trip Start Timestamp'] = chicago_trips['Trip Start Timestamp'].dt.tz_localize('UTC')

# 3. Floor to the hour to match weather granularity
chicago_trips['Trip Start Hour'] = chicago_trips['Trip Start Timestamp'].dt.floor('H')

# 4. Merge on floored timestamp and weather timestamp
merged = chicago_trips.merge(
    chicago_weather,
    how='left',
    left_on='Trip Start Hour',
    right_on='timestamp_utc'
)

C:\Users\Anthony\AppData\Local\Temp\ipykernel_1952\2716001261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chicago_weather['timestamp_utc'] = pd.to_datetime(chicago_weather['timestamp_utc'], utc=True)


In [15]:
chicago_2024 = merged[merged['Trip Start Timestamp'].dt.year == 2024]

In [16]:
chicago_2024.reset_index(drop=True)

Trip Start Timestamp  Trip End Timestamp  Trip Seconds  \
0       2024-01-19 17:00:00+00:00 2024-01-19 18:00:00        4051.0   
1       2024-01-05 09:00:00+00:00 2024-01-05 09:00:00         517.0   
2       2024-01-18 19:15:00+00:00 2024-01-18 19:30:00        1004.0   
3       2024-01-04 07:15:00+00:00 2024-01-04 07:30:00         484.0   
4       2024-01-25 11:15:00+00:00 2024-01-25 11:30:00        1686.0   
...                           ...                 ...           ...   
4888772 2024-12-31 16:15:00+00:00 2024-12-31 16:45:00        2088.0   
4888773 2024-12-31 16:45:00+00:00 2024-12-31 17:15:00        2017.0   
4888774 2024-12-31 11:30:00+00:00 2024-12-31 11:45:00        1500.0   
4888775 2024-12-31 15:15:00+00:00 2024-12-31 16:00:00        2520.0   
4888776 2024-12-31 14:00:00+00:00 2024-12-31 14:45:00        2400.0   

         Trip Miles   Fare   Tips  Tolls  Extras  Trip Total  \
0             17.12  45.50  10.00    0.0     4.0       60.00   
1              3.39  10.91   2.78    0.0     1.0       14.69   
2              1.18  15.94   3.72    0.0     0.0       19.66   
3              1.59   7.75   1.24    0.0     0.0        9.49   
4             13.01  34.25   7.95    0.0     5.0       47.70   
...             ...    ...    ...    ...     ...         ...   
4888772       12.14  33.00  11.25    0.0     4.0       48.75   
4888773       12.58  34.25   8.05    0.0     5.5       48.30   
4888774        7.80  23.50   0.00    0.0     0.0       23.50   
4888775        8.90  28.75   0.00    0.0     0.0       28.75   
4888776       15.60  40.25  10.00    0.0     4.0       54.25   

         Pickup Centroid Latitude  ...             timestamp_utc  temp  \
0                       41.979071  ... 2024-01-19 17:00:00+00:00 -10.7   
1                       41.944227  ... 2024-01-05 09:00:00+00:00  -0.9   
2                       41.880994  ... 2024-01-18 19:00:00+00:00  -4.6   
3                       41.879255  ... 2024-01-04 07:00:00+00:00   1.4   
4                       41.884987  ... 2024-01-25 11:00:00+00:00   1.8   
...                           ...  ...                       ...   ...   
4888772                 41.792592  ... 2024-12-31 16:00:00+00:00   2.8   
4888773                 41.980264  ... 2024-12-31 16:00:00+00:00   2.8   
4888774                 41.899602  ... 2024-12-31 11:00:00+00:00   3.8   
4888775                 41.954028  ... 2024-12-31 15:00:00+00:00   2.8   
4888776                 41.979071  ... 2024-12-31 14:00:00+00:00   3.4   

         app_temp    rh wind_spd wind_gust_spd  precip snow    ghi   vis  
0           -19.5  68.0     6.70          12.8    0.00  0.0  426.0  10.0  
1            -4.2  69.0     2.60           3.5    0.00  0.0    0.0  16.0  
2            -9.6  65.0     3.60           3.6    0.00  0.0  424.0  16.0  
3            -2.6  71.0     4.09           7.5    0.00  0.0    0.0  16.0  
4            -1.8  96.0     3.60           4.6    0.00  0.0    0.0  10.0  
...           ...   ...      ...           ...     ...  ...    ...   ...  
4888772      -2.4  91.0     7.20          13.6    2.50  0.0  311.0  10.0  
4888773      -2.4  91.0     7.20          13.6    2.50  0.0  311.0  10.0  
4888774      -0.9  91.0     6.70          14.0    0.25  0.0    0.0  10.0  
4888775      -2.6  89.0     7.70          14.0    2.00  0.0  196.0  10.0  
4888776      -1.6  88.0     7.20          14.4    2.00  0.0   62.0  10.0  

[4888777 rows x 28 columns]

In [18]:
chicago_2024.to_csv("Chicago_Trips_Weather_2024.csv", index=False)